<a href="https://colab.research.google.com/github/Timidfault72/OOU_CSC309_Artificial_Intelligence/blob/main/notebooks/CSC309_Week05_KRR_Logic_Student_Centred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC309 – Artificial Intelligence  
**Week 5 Lab:** Knowledge Representation & Reasoning — Logic & Inference

**Instructor:** Dr Sakinat Folorunso

**Title:** Associate Professor of AI Systems and FAIR Data **Department:** Computer Sciences, Olabisi Onabanjo University, Ago-Iwoye, Ogun State, Nigeria

**Course Code:** CSC 309

**Mode:** Student‑centred, hands‑on in Google Colab

> Every code cell is commented line‑by‑line so you can follow the logic precisely.

## How to use this notebook
1. Start with the **Group Log** and **Do Now**.  
2. Run the **Setup** cell once.  
3. Work through **Tasks**. Edit only cells marked **`# TODO(Student)`**.  
4. Use **Quick Checks** to test your understanding.  
5. Finish with the **Reflection**. If you finish early, try the **Extensions**.

In [6]:
#@title 🧑🏽‍🤝‍🧑🏾 Group Log (fill before you start)
# The '#@param' annotations create form fields in Colab for easy input.

group_members = "Ogunkoya Ibrahim, Ayodele Shaahid, Adekanbi Akorede Elijah, Ogunjirin Olamide, Akapo Samuel, Aina Prosperity, Ayodele Shaahid"  #@param {type:"string"}  # Names of teammates
roles_notes = "Driver/Navigator, decisions, questions"  #@param {type:"string"}  # Short working notes

print("👥 Group:", group_members)        # Echo the group list for confirmation
print("📝 Notes:", roles_notes)          # Echo the notes so they're preserved in output

👥 Group: Ogunkoya Ibrahim, Ayodele Shaahid, Adekanbi Akorede Elijah, Ogunjirin Olamide, Akapo Samuel, Aina Prosperity, Ayodele Shaahid
📝 Notes: Driver/Navigator, decisions, questions


### Learning Objectives
- Encode facts/rules in a small **logic knowledge base**.  
- Query using unification; practice NL→logic, CNF/resolution (by hand).

In [7]:
#@title 🔧 Setup
# We try to use 'kanren' for a taste of relational programming.
# If unavailable, we fall back to a tiny custom rule demo.

import sys, subprocess                                     # Tools for optional installs
def pip_install(pkgs):
    for p in pkgs:
        try: __import__(p.split("==")[0])                  # Try importing
        except Exception:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", p])  # Install if missing

try:
    pip_install(["kanren", "unification", "toolz"])        # Attempt to install kanren ecosystem
    from kanren import Relation, facts, run, var           # Import relational primitives
    using_kanren = True                                     # Flag to indicate the backend in use
except Exception as e:
    print("Using simple fallback (kanren not available):", e) # Inform the student
    using_kanren = False                                    # Switch to fallback
print("✅ Setup complete for Week 5.")

✅ Setup complete for Week 5.


In [8]:
#@title 📐 Starter KB (line‑by‑line comments)

if 'using_kanren' in globals() and using_kanren:    # If kanren is available
    parent = Relation()                              # Declare a binary relation 'parent(x,y)'
    facts(parent, ("ada","sam"), ("sam","tayo"), ("sam","ife"), ("ife","bola"))  # Add some facts
    x = var()                                        # Logic variable 'x'
    y = var()                                        # Logic variable 'y'
    # Query: who are Ada's grandchildren?
    result = run(0, x, parent("ada", x), parent(x, y))   # Find x such that parent(ada,x) and parent(x,y)
    print("Grandchildren of Ada (via kanren):", set(result))  # Show solutions
else:
    # Fallback: a tiny Python-based relation
    PARENT = {("ada","sam"), ("sam","tayo"), ("sam","ife"), ("ife","bola")}  # Parent relation as a set of tuples
    def grandchildren(p):                 # Compute grandchildren of 'p' by relational join
        return {c2 for (p1,c1) in PARENT for (p2,c2) in PARENT if p1==p and p2==c1}
    print("Grandchildren of Ada (fallback):", grandchildren("ada"))  # Show result

Grandchildren of Ada (via kanren): {'sam'}


In [9]:
# TODO(Student): Encode 'All humans are mortal; Socrates is human; therefore Socrates is mortal.'
# Option A (kanren): Create relations HUMAN(x) and MORTAL(x); add facts and rules; run a query for Socrates.
from kanren import Relation, facts, run, var

# Create relations
HUMAN = Relation()
MORTAL = Relation()

# Add facts
facts(HUMAN, ("socrates",))
# Rule: all humans are mortal (we encode this by asserting mortality for every human)
facts(MORTAL, ("socrates",))  # derived manually since kanren doesn't do rules automatically

x = var()

# Query: is Socrates mortal?
result = run(1, x, MORTAL(x))
print(result)

# Option B (fallback): Represent as Python sets/dicts and derive 'mortal(socrates)' procedurally.
# Facts
human = {"socrates"}
mortal = set()

# Rule: if someone is human, they are mortal
for x in human:
    mortal.add(x)

# Query
print("socrates" in mortal)  # True


('socrates',)
True


### Quick Check
Translate **two** everyday constraints into predicate logic, then back into English.  
E.g., “All 300‑level courses have a CA in Week 7.”

1.

let
300-levelcourses(x) = "x is a student"

CA in week 7(x) = "x has a CA in week 7"

Formula:

∀𝑥 (300-levelcourses(𝑥)→𝐻𝑎𝑠CAinweek7(𝑥))∀x(300levelcourses(x)→HasCAin week 7(x))

Back to English

“All 300‑level courses have a CA in Week 7.”

2.

Let
Student(x) = “x is a student”

HasID(x) = “x has a valid ID card”

Formula:
∀𝑥 (𝑆𝑡𝑢𝑑e𝑛𝑡(𝑥)→𝐻𝑎𝑠𝐼𝐷(𝑥))∀x(Student(x)→HasID(x))

Back to English

“Every student must have a valid ID card.”